### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/310_TROCAS7_Fusion_Apr2021_PEAKS_127/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/310_TROCAS7_Fusion_Apr2021_PEAKS_127


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 310_TROCAS7_Fusion_Apr2021_PEAKS_127_peptide.csv
 310_TROCAS7_Fusion_Apr2021_PEAKS_127_protein.csv
 310_TROCAS7_Fusion_Apr2021_PEAKS_127_protein.fasta
'DB search psm.csv'
'de novo only peptides.csv'
 peptides.pep.xml
 protein-peptides.csv


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup310 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/310_TROCAS7_Fusion_Apr2021_PEAKS_127/310_TROCAS7_Fusion_Apr2021_PEAKS_127_peptide.csv")

# remove redundant rows
peaksdb310 = pd.DataFrame.drop_duplicates(peaksdbdup310)

print(peaksdb310.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb310.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb310['#Spec']
del peaksdb310['Accession']
del peaksdb310['PTM']
del peaksdb310['AScore']

mean_length = peaksdb310['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup310))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb310))

#look at the dataframe
peaksdb310.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Trocas7-310-SMCP', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Trocas7-310-SMCP', 'Accession', 'PTM', 'AScore'],
      dtype='object')
11.156862745098039
# redundant peaksdb peptides in combined dataframe 51
# nonredundant peaksdb peptides in combined dataframe 51


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,76.36,1881.8350,18,0.8,941.9255,92.53,1730000.0,3,15045,20210406_Trocas7_652_SMCP310_DDA_120min_1.raw
1,SSGSSYPSLLQC(+57.02)LK,60.40,1525.7446,14,1.7,763.8809,100.61,367000.0,3,16433,20210406_Trocas7_652_SMCP310_DDA_120min_1.raw
2,LGEHNIDVLEGNEQFINAAK,54.36,2210.0967,20,0.2,737.7063,95.24,247000.0,3,15527,20210406_Trocas7_652_SMCP310_DDA_120min_1.raw
3,SC(+57.02)AAAGTEC(+57.02)LISGWGN,45.04,1652.6923,16,-0.2,827.3533,99.97,237000.0,3,16316,20210406_Trocas7_652_SMCP310_DDA_120min_1.raw
4,APVLSDSSC(+57.02)K,43.65,1062.5016,10,0.3,532.2582,36.18,198000.0,3,5692,20210406_Trocas7_652_SMCP310_DDA_120min_1.raw


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb310['A'] = peaksdb310['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb310['C'] = peaksdb310['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb310['D'] = peaksdb310['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb310['E'] = peaksdb310['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb310['F'] = peaksdb310['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb310['G'] = peaksdb310['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb310['H'] = peaksdb310['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb310 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb310['I'] = peaksdb310['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb310['K'] = peaksdb310['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb310['L'] = peaksdb310['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb310['M'] = peaksdb310['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb310['N'] = peaksdb310['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb310['P'] = peaksdb310['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb310['Q'] = peaksdb310['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb310['R'] = peaksdb310['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb310['S'] = peaksdb310['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb310['T'] = peaksdb310['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb310['V'] = peaksdb310['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb310['W'] = peaksdb310['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb310['Y'] = peaksdb310['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb310['c-carb'] = peaksdb310['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb310['m-oxid'] = peaksdb310['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb310['n-deam'] = peaksdb310['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb310['q-deam'] = peaksdb310['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb310['stripped_peptide'] = peaksdb310['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb310['stripped_length'] = peaksdb310['stripped_peptide'].apply(len)

peaksdb310['NAAF_num.'] = peaksdb310['Area'] / peaksdb310['stripped_length']

# total the number of modifications in sequence
peaksdb310['ptm-total'] = peaksdb310['c-carb'] + peaksdb310['m-oxid'] + peaksdb310['n-deam'] + peaksdb310['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb310['stripped_IL']= peaksdb310['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb310.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/310_MS_T24_GF_PDB/310A_MS_T24_GF_PDB.csv")

# check out the results
peaksdb310.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,76.36,1881.8350,18,0.8,941.9255,92.53,1730000.0,3,15045,...,0,2,0,0,0,SCLISGWGNTK,11,157272.727273,2,SCLLSGWGNTK
1,SSGSSYPSLLQC(+57.02)LK,60.40,1525.7446,14,1.7,763.8809,100.61,367000.0,3,16433,...,1,1,0,0,0,SSGSSYPSLLQCLK,14,26214.285714,1,SSGSSYPSLLQCLK
2,LGEHNIDVLEGNEQFINAAK,54.36,2210.0967,20,0.2,737.7063,95.24,247000.0,3,15527,...,0,0,0,0,0,LGEHNIDVLEGNEQFINAAK,20,12350.000000,0,LGEHNLDVLEGNEQFLNAAK
3,SC(+57.02)AAAGTEC(+57.02)LISGWGN,45.04,1652.6923,16,-0.2,827.3533,99.97,237000.0,3,16316,...,0,2,0,0,0,SCLISGWGN,9,26333.333333,2,SCLLSGWGN
4,APVLSDSSC(+57.02)K,43.65,1062.5016,10,0.3,532.2582,36.18,198000.0,3,5692,...,0,1,0,0,0,APVLSDSSCK,10,19800.000000,1,APVLSDSSCK


In [6]:
# keep only stripped peptide I/L and NAAF
db_310 = peaksdb310[['stripped_IL', 'Area', 'NAAF_num.']]

db_310.set_index('stripped_IL')

# write modified dataframe to new txt file
db_310.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/310A_MS_T24_GF_PDB_ILnaafs.csv")

db_310.head()

,stripped_IL,Area,NAAF_num.
0,SCLLSGWGNTK,1730000.0,157272.727273
1,SSGSSYPSLLQCLK,367000.0,26214.285714
2,LGEHNLDVLEGNEQFLNAAK,247000.0,12350.000000
3,SCLLSGWGN,237000.0,26333.333333
4,APVLSDSSCK,198000.0,19800.000000


### Exporting txt files of stripped peptides at confidence cutoffs:

In [7]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep310moddup = peaksdb310[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep310dup = peaksdb310[["stripped_peptide"]]

# deduplicate both of these lists
pep310mod = pep310moddup.drop_duplicates()
pep310 = pep310dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 310:", len(pep310moddup))
print("Deduplicated modified PeaksDB peptides in 310:", len(pep310mod))
print("Total stripped PeaksDB peptides in 310:", len(pep310dup))
print("Deduplicated stripped PeaksDB peptides in 310:", len(pep310))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep310.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/310_MS_T24_GF_PDB/310A_MS_T24_GF_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/310_MS_T24_GF_PDB/310A_MS_T24_GF_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/310_MS_T24_GF_PDB/310A_MS_T24_GF_PDB_stripped_peptides.fas

# look at the stripped peptides
pep310.head()

Total modified PeaksDB peptides in 310: 51
Deduplicated modified PeaksDB peptides in 310: 51
Total stripped PeaksDB peptides in 310: 51
Deduplicated stripped PeaksDB peptides in 310: 50


,stripped_peptide
0,SCLISGWGNTK
1,SSGSSYPSLLQCLK
2,LGEHNIDVLEGNEQFINAAK
3,SCLISGWGN
4,APVLSDSSCK


## NAAF correction and exporting files with AA and PTM totals:

In [12]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb310['A'].sum(),
        'C': peaksdb310['C'].sum(),
        'D': peaksdb310['D'].sum(),
        'E': peaksdb310['E'].sum(),
        'F': peaksdb310['F'].sum(),
        'G': peaksdb310['G'].sum(),
        'H': peaksdb310['H'].sum(),
        'I': peaksdb310['I'].sum(),
        'K': peaksdb310['K'].sum(),
        'L': peaksdb310['L'].sum(),
        'M': peaksdb310['M'].sum(),
        'N': peaksdb310['N'].sum(),
        'P': peaksdb310['P'].sum(),
        'Q': peaksdb310['Q'].sum(),
        'R': peaksdb310['R'].sum(),
        'S': peaksdb310['S'].sum(),
        'T': peaksdb310['T'].sum(),
        'V': peaksdb310['V'].sum(),
        'W': peaksdb310['W'].sum(),
        'Y': peaksdb310['Y'].sum(),
        'c-carb': peaksdb310['c-carb'].sum(),
        'm-oxid': peaksdb310['m-oxid'].sum(),
        'n-deam': peaksdb310['n-deam'].sum(),
        'q-deam': peaksdb310['q-deam'].sum(),
        'Total area': peaksdb310['Area'].sum(),
        'Total length': peaksdb310['stripped_length'].sum()
       }

totalpeaksdb310 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb310['% C w/ carb'] = totalpeaksdb310['c-carb'] / totalpeaksdb310['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb310['% M w/ oxid'] = totalpeaksdb310['m-oxid'] / totalpeaksdb310['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb310['% N w/ deam'] = totalpeaksdb310['n-deam'] / totalpeaksdb310['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb310['% Q w/ deam'] = totalpeaksdb310['q-deam'] / totalpeaksdb310['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb310['NAAF denom.'] = totalpeaksdb310['Total area'] / totalpeaksdb310['Total length']

# write modified dataframe to new txt file
totalpeaksdb310.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/310_MS_T24_GF_PDB/310A_MS_T24_GF_PDB_totals.csv")

totalpeaksdb310.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,36,12,11,27,7,81,12,34,23,58,...,1,4,3,183254116.0,520,1.0,0.333333,0.121212,0.15,352411.761538


In [13]:
# use the calculated NAAF factor (in totalpeaksdb310 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 352411.761538

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb310['NAAF factor'] = (peaksdb310['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb310_NAAF = peaksdb310[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb310_NAAF['A-NAAF20'] = peaksdb310_NAAF['A'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['C-NAAF20'] = peaksdb310_NAAF['C'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['D-NAAF20'] = peaksdb310_NAAF['D'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['E-NAAF20'] = peaksdb310_NAAF['E'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['F-NAAF20'] = peaksdb310_NAAF['F'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['G-NAAF20'] = peaksdb310_NAAF['G'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['H-NAAF20'] = peaksdb310_NAAF['H'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['I-NAAF20'] = peaksdb310_NAAF['I'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['K-NAAF20'] = peaksdb310_NAAF['K'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['L-NAAF20'] = peaksdb310_NAAF['L'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['M-NAAF20'] = peaksdb310_NAAF['M'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['N-NAAF20'] = peaksdb310_NAAF['N'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['P-NAAF20'] = peaksdb310_NAAF['P'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['Q-NAAF20'] = peaksdb310_NAAF['Q'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['R-NAAF20'] = peaksdb310_NAAF['R'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['S-NAAF20'] = peaksdb310_NAAF['S'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['T-NAAF20'] = peaksdb310_NAAF['T'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['V-NAAF20'] = peaksdb310_NAAF['V'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['W-NAAF20'] = peaksdb310_NAAF['W'] * peaksdb310['NAAF factor']
peaksdb310_NAAF['Y-NAAF20'] = peaksdb310_NAAF['Y'] * peaksdb310['NAAF factor']

# multiply the NAAF20 factor by the PTMs normalize its abundance by peak area and peptide length

peaksdb310_NAAF['ccarb-NAAF20'] = peaksdb310_NAAF['c-carb'] * peaksdb310_NAAF['NAAF factor']
peaksdb310_NAAF['moxid-NAAF20'] = peaksdb310_NAAF['m-oxid'] * peaksdb310_NAAF['NAAF factor']
peaksdb310_NAAF['ndeam-NAAF20'] = peaksdb310_NAAF['n-deam'] * peaksdb310_NAAF['NAAF factor']
peaksdb310_NAAF['qdeam-NAAF20'] = peaksdb310_NAAF['q-deam'] * peaksdb310_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb310_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/310_MS_T24_GF_PDB/310A_MS_T24_GF_PDB_naaf.csv")

peaksdb310_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,SCLISGWGNTK,0.446275,3,2,0,1,0,3,0,1,...,0.0,0.892551,0.892551,0.000000,0.446275,0.000000,0.892551,0.0,0.0,0.0
1,SSGSSYPSLLQCLK,0.074385,0,1,0,0,0,1,0,0,...,0.0,0.371927,0.000000,0.000000,0.000000,0.074385,0.074385,0.0,0.0,0.0
2,LGEHNIDVLEGNEQFINAAK,0.035044,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,0.035044,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,SCLISGWGN,0.074723,3,2,0,1,0,3,0,1,...,0.0,0.149446,0.074723,0.000000,0.074723,0.000000,0.149446,0.0,0.0,0.0
4,APVLSDSSCK,0.056184,1,1,1,0,0,0,0,0,...,0.0,0.168553,0.000000,0.056184,0.000000,0.000000,0.056184,0.0,0.0,0.0


In [14]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb310 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb310_NAAF['NAAF factor'].sum(),
        'A': peaksdb310_NAAF['A-NAAF20'].sum(),
        'C': peaksdb310_NAAF['C-NAAF20'].sum(),
        'D': peaksdb310_NAAF['D-NAAF20'].sum(),
        'E': peaksdb310_NAAF['E-NAAF20'].sum(),
        'F': peaksdb310_NAAF['F-NAAF20'].sum(),
        'G': peaksdb310_NAAF['G-NAAF20'].sum(),
        'H': peaksdb310_NAAF['H-NAAF20'].sum(),
        'I': peaksdb310_NAAF['I-NAAF20'].sum(),
        'K': peaksdb310_NAAF['K-NAAF20'].sum(),
        'L': peaksdb310_NAAF['L-NAAF20'].sum(),
        'M': peaksdb310_NAAF['M-NAAF20'].sum(),
        'N': peaksdb310_NAAF['N-NAAF20'].sum(),
        'P': peaksdb310_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb310_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb310_NAAF['R-NAAF20'].sum(),
        'S': peaksdb310_NAAF['S-NAAF20'].sum(),
        'T': peaksdb310_NAAF['T-NAAF20'].sum(),
        'V': peaksdb310_NAAF['V-NAAF20'].sum(),
        'W': peaksdb310_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb310_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb310_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb310_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb310_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb310_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb310_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb310_NAAF['% C w/ carb.'] = totalpeaksdb310_NAAF['c-carb'] / totalpeaksdb310_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb310_NAAF['% M w/ oxid'] = totalpeaksdb310_NAAF['m-oxid'] / totalpeaksdb310_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb310_NAAF['% N w/ deam'] = totalpeaksdb310_NAAF['n-deam'] / totalpeaksdb310_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb310_NAAF['% Q w/ deam'] = totalpeaksdb310_NAAF['q-deam'] / totalpeaksdb310_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb310_NAAF['NAAF check'] = totalpeaksdb310_NAAF['NAAF'] / 352411.761538

# write modified dataframe to new txt file, same name + totals
totalpeaksdb310_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/310_MS_T24_GF_PDB/310A_MS_T24_GF_PDB_naaf_totals.csv")

totalpeaksdb310_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,60.442722,46.297336,1.992912,0.77,1.330342,0.163811,6.820721,2.143875,2.656515,3.003781,...,1.807998,1.992912,0.012627,0.117994,0.00074,1.0,0.007822,0.009011,0.000173,0.000172
